In [ ]:
from typing import List

class View:
    def __init__(self, sheet):
        self.sheet = sheet

    def get_cells(self):
        raise NotImplemented

class SingleCell(View):
    def __init__(self, sheet, location):
        super().__init__(sheet)
        self.location = location
    
    def get_cells(self):
        r, c = self.location
        if f'{r}_{c}' not in self.sheet.data:
            self.sheet.set(r, c, 0)
            
        return [self.sheet.data[f'{r}_{c}']]

class Collection(View):
    def __init__(self, sheet, subviews: List[View]):
        super().__init__(sheet)
        self.subviews = subviews

    def get_cells(self):
        ret = []

        for view in self.subviews:
            ret += view.get_cells()

        return ret

class Range(View):
    def __init__(self, top_left, bottom_right, sheet):
        super().__init__(sheet)
        self.top_left = top_left
        self.bottom_right = bottom_right

    def get_cells(self):
        rtl, ctl = self.top_left
        rbr, cbr = self.bottom_right

        ret = []
        for r in range(rtl, rbr+1):
            for c in range(ctl, cbr+1):
                if f'{r}_{c}' not in self.sheet.data:
                    self.sheet.set(r, c, 0)
                
                ret.append(self.sheet.data[f'{r}_{c}'])

        return ret


class Expr:
    def __init__(self, input_view: View):
        def throw():
           raise NotImplemented

        self.aggr = throw
        self.upstreams = input_view

    def eval(self):
        params = [cell.getVal() for cell in self.upstreams.get_cells()]
        return self.aggr(params)

class Sum(Expr):
    def __init__(self, input_view):
        super().__init__(input_view)
        self.aggr = sum


class Cell:
    def __init__(self, sheet, location):
        self.sheet = sheet
        self.expr = None
        self.val = None
        self.loc = location
        self.downstreams = set()

    def __hash__(self):
        return hash(f'{self.sheet.name}__{self.loc}')

    def getVal(self):
        return self.val

    def _remove_expr(self):
        if self.expr is not None:
            for cell in self.expr.upstreams.get_cells():
                cell.downstreams.remove(self)

            self.expr = None   


    def setVal(self, data):
        curr_val = self.val

        if isinstance(data, float) or isinstance(data, int):
            self.val = data
            self._remove_expr()
             

        elif isinstance(data, Expr):
            # TODO: cyclic dependency detection
            self._remove_expr()

            self.expr = data
            for cell in self.expr.upstreams.get_cells():
                cell.downstreams.add(self)
            self.val = data.eval()
        else:
            raise NotImplemented

        if curr_val != self.val:
            for cell in self.downstreams:
                cell.update(delta = self.val-curr_val)

    def update(self):
        assert self.expr is not None, 'downstream node supposed to be expr'
        self.val = self.expr.eval()  
        for cell in self.downstreams:
            cell.update()

class MySheet:
    def __init__(self, name='new_sheet'):
        self.name = name
        self.data = {}

    def set(self, row, col, val):
        key = f'{row}_{col}'
        if key not in self.data:
            self.data[key] = Cell(self, key)

        self.data[key].setVal(val)

    def get(self, row, col):
        if f'{row}_{col}' in self.data:
            return self.data[f'{row}_{col}'].getVal()
        else:
            return None


In [ ]:
sheet = MySheet()

sheet.set(1, 1, 2)
assert sheet.get(1, 1) == 2

sheet.set(1, 2, 3)

expr = Sum(Range((1, 1), (1, 5), sheet))
assert expr.eval() == 5

sheet.set(2, 1, expr)
assert sheet.get(2, 1) == 5

sheet.set(1, 3, -2)
assert sheet.get(2, 1) == 3

expr2 = Sum(Collection(sheet, [SingleCell(sheet, (2, 1)), Range((3, 1), (4, 2), sheet)])) 

sheet.set(2, 2, expr2)
assert sheet.get(2, 2) == 3

sheet.set(3, 1, 0.5)
assert sheet.get(2, 2) == 3.5

sheet.set(4, 1, 0.3)
assert sheet.get(2, 2) == 3.8

sheet.set(1, 3, 0)
assert sheet.get(2, 1) == 5
assert sheet.get(2, 2) == 5.8

sheet.set(3, 1, expr)
assert sheet.get(2, 2) == 10.3

sheet.set(3, 1, 4)
assert sheet.get(2, 2) == 9.3

sheet.set(3, 1, expr)
assert sheet.get(2, 2) == 10.3

sheet.set(3, 1, 5)
assert sheet.get(2, 2) == 10.3

